# Problema Abordado

# Implementação

## Importações

In [1]:
import torch
from torchvision import transforms, datasets
from torchvision.models import models
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
from PIL import Image
import os






ImportError: cannot import name 'models' from 'torchvision.models' (C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\__init__.py)

##Funções Auxiliares

In [ ]:
def train_model(
    model,          # Seu modelo
    train_loader,   # DataLoader de treino
    criterion,      # Função de perda
    optimizer,      # Otimizador
    device,         # "cuda" ou "cpu"
    epochs=10       # Número de épocas
):
    model.to(device)
    model.train()  # Modo treino

    for epoch in range(epochs):
        epoch_loss = 0.0

        for images, captions in train_loader:

            images = images.to(device)
            captions = captions.to(device)


            optimizer.zero_grad()


            outputs = model(images, captions[:, :-1])
            loss = criterion(
                outputs.view(-1, outputs.size(-1)),
                captions[:, 1:].reshape(-1)
            )

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        # Loss médio da época
        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)), # Padrão dos modelos pretreinados do ImageNet
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406],  ## Normalizando dados no padrão do ImageNet
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
# def buildDeepFashionDataset(labelDataset):

#     for path in labelDataset['path']:
#         img_path = f'/datasets/selected_images/{path}'
#         image = Image.open(img_path).convert("RGB")
#         transformed_image = transform(image)
        
#         print(transformed_image)
    

## Dados

###Carregando o dataset


In [65]:
class DeepFashionDataset(Dataset):
    def __init__(self, labelDataset, image_dir, transform=None, tokenizer=None, max_length=30):
        self.df = labelDataset.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # --- Imagem ---
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['path'])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        # --- Texto ---
        caption = row['caption']
        if self.tokenizer:
            tokens = self.tokenizer(caption, padding="max_length", truncation=True,
                                    max_length=self.max_length, return_tensors="pt")
            input_ids = tokens["input_ids"].squeeze(0)
            attention_mask = tokens["attention_mask"].squeeze(0)
        else:
            input_ids = caption
            attention_mask = None

        return {
            "image": image,
            "input_ids": 'input_ids',
            "attention_mask": 'attention_mask'
        }


In [66]:
labels_df = pd.read_csv('datasets/labels_front.csv')
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12278 entries, 0 to 12277
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   image_id      12278 non-null  object
 1   caption       12278 non-null  object
 2   path          12278 non-null  object
 3   gender        12278 non-null  object
 4   product_type  12278 non-null  object
 5   product_id    12278 non-null  object
 6   image_type    12278 non-null  object
dtypes: object(7)
memory usage: 671.6+ KB


### Data Loader

In [67]:
dataset = DeepFashionDataset(
    labelDataset = labels_df,
    image_dir = "datasets/selected_images",
    transform = transform
    # tokenizer = tokenizer
)


In [68]:
batch_size = 64
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# img_path = 'datasets/selected_images/MEN-Denim-id_00000089-28_1_front.jpg'
# image = Image.open(img_path).convert("RGB")

# transform = transforms.Compose([
#     transforms.Resize((224,224)), # Padrão dos modelos pretreinados de CNN do ImageNet. Esse tamanho pode ser modificado com base na nossa CNN
#     transforms.ToTensor(),  # Converte para tensor [C, H, W] , float32 e normalizado [0, 1]
#     transforms.Normalize(mean = [0.485, 0.456, 0.406],  ## Normalizando dados no padrão do ImageNet
#                          std=[0.229, 0.224, 0.225])
# ])

# print(transform(image)) #aplicando transformações e adiciona dimensão de batch: shape vira (1, 3, 224, 224)

 

## Rede Implementada

In [ ]:


class ImageToText(nn.Module):
    def __init__(self):
        super().__init__()

        # Encoder (CNN - Custom)
        super().__init__()
        
        self.encoder = models.resnet50(pretrained=True)
        self.encoder.fc = nn.Identity()  
        


        # Decoder (GPT2)


    def forward(self,):


# Treinamento da Rede

# Qualidade dos Resultados

# Discussão Geral